In [0]:
#Wykorzystaj przykłady z notatnika w SQL Windowed Aggregate Functions (cmd 11) i przepisz funkcje używając Spark API.  


In [0]:
%sql
Create database if not exists Sample

In [0]:
%sql

CREATE TABLE IF NOT EXISTS Sample.Transactions ( AccountId INT, TranDate DATE, TranAmt DECIMAL(8, 2));

CREATE TABLE IF NOT EXISTS Sample.Logical (RowID INT,FName VARCHAR(20), Salary SMALLINT);

In [0]:
%sql

INSERT INTO Sample.Transactions VALUES 
( 1, '2011-01-01', 500),
( 1, '2011-01-15', 50),
( 1, '2011-01-22', 250),
( 1, '2011-01-24', 75),
( 1, '2011-01-26', 125),
( 1, '2011-01-28', 175),
( 2, '2011-01-01', 500),
( 2, '2011-01-15', 50),
( 2, '2011-01-22', 25),
( 2, '2011-01-23', 125),
( 2, '2011-01-26', 200),
( 2, '2011-01-29', 250),
( 3, '2011-01-01', 500),
( 3, '2011-01-15', 50 ),
( 3, '2011-01-22', 5000),
( 3, '2011-01-25', 550),
( 3, '2011-01-27', 95 ),
( 3, '2011-01-30', 2500)



num_affected_rows,num_inserted_rows
18,18


In [0]:
%sql
INSERT INTO Sample.Logical
VALUES (1,'George', 800),
(2,'Sam', 950),
(3,'Diane', 1100),
(4,'Nicholas', 1250),
(5,'Samuel', 1250),
(6,'Patricia', 1300),
(7,'Brian', 1500),
(8,'Thomas', 1600),
(9,'Fran', 2450),
(10,'Debbie', 2850),
(11,'Mark', 2975),
(12,'James', 3000),
(13,'Cynthia', 3000),
(14,'Christopher', 5000);

num_affected_rows,num_inserted_rows
14,14


Totals based on previous row

In [0]:
%sql

SELECT AccountId,
TranDate,
TranAmt,
-- running total of all transactions
SUM(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate) as RunTotalAmt
FROM Sample.Transactions ORDER BY AccountId, TranDate;

AccountId,TranDate,TranAmt,RunTotalAmt
1,2011-01-01,500.00,500.00
1,2011-01-15,50.00,550.00
1,2011-01-22,250.00,800.00
1,2011-01-24,75.00,875.00
1,2011-01-26,125.00,1000.00
1,2011-01-28,175.00,1175.00
2,2011-01-01,500.00,500.00
2,2011-01-15,50.00,550.00
2,2011-01-22,25.00,575.00
2,2011-01-23,125.00,700.00


In [0]:
%sql
SELECT AccountId,
TranDate,
TranAmt,
-- running average of all transactions
AVG(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate) as RunAvg,
-- running total # of transactions
COUNT(*) OVER (PARTITION BY AccountId ORDER BY TranDate) as RunTranQty,
-- smallest of the transactions so far
MIN(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate) as RunSmallAmt,
-- largest of the transactions so far
MAX(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate) as RunLargeAmt,
-- running total of all transactions
SUM(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate) RunTotalAmt
FROM Sample.Transactions 
ORDER BY AccountId,TranDate;

AccountId,TranDate,TranAmt,RunAvg,RunTranQty,RunSmallAmt,RunLargeAmt,RunTotalAmt
1,2011-01-01,500.00,500.000000,1,500.00,500.00,500.00
1,2011-01-15,50.00,275.000000,2,50.00,500.00,550.00
1,2011-01-22,250.00,266.666667,3,50.00,500.00,800.00
1,2011-01-24,75.00,218.750000,4,50.00,500.00,875.00
1,2011-01-26,125.00,200.000000,5,50.00,500.00,1000.00
1,2011-01-28,175.00,195.833333,6,50.00,500.00,1175.00
2,2011-01-01,500.00,500.000000,1,500.00,500.00,500.00
2,2011-01-15,50.00,275.000000,2,50.00,500.00,550.00
2,2011-01-22,25.00,191.666667,3,25.00,500.00,575.00
2,2011-01-23,125.00,175.000000,4,25.00,500.00,700.00


* Calculating Totals Based Upon a Subset of Rows

In [0]:
%sql
SELECT AccountId,
TranDate,
TranAmt,
-- average of the current and previous 2 transactions
AVG(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as SlideAvg,
-- total # of the current and previous 2 transactions
COUNT(*) OVER (PARTITION BY AccountId ORDER BY TranDate ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as SlideQty,
-- smallest of the current and previous 2 transactions
MIN(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as SlideMin,
-- largest of the current and previous 2 transactions
MAX(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as SlideMax,
-- total of the current and previous 2 transactions
SUM(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as SlideTotal,
ROW_NUMBER() OVER (PARTITION BY AccountId ORDER BY TranDate) AS RN
FROM Sample.Transactions 
ORDER BY AccountId, TranDate, RN

AccountId,TranDate,TranAmt,SlideAvg,SlideQty,SlideMin,SlideMax,SlideTotal,RN
1,2011-01-01,500.00,500.000000,1,500.00,500.00,500.00,1
1,2011-01-15,50.00,275.000000,2,50.00,500.00,550.00,2
1,2011-01-22,250.00,266.666667,3,50.00,500.00,800.00,3
1,2011-01-24,75.00,125.000000,3,50.00,250.00,375.00,4
1,2011-01-26,125.00,150.000000,3,75.00,250.00,450.00,5
1,2011-01-28,175.00,125.000000,3,75.00,175.00,375.00,6
2,2011-01-01,500.00,500.000000,1,500.00,500.00,500.00,1
2,2011-01-15,50.00,275.000000,2,50.00,500.00,550.00,2
2,2011-01-22,25.00,191.666667,3,25.00,500.00,575.00,3
2,2011-01-23,125.00,66.666667,3,25.00,125.00,200.00,4


* Logical Window


In [0]:
%sql
SELECT RowID,
FName,
Salary,
SUM(Salary) OVER (ORDER BY Salary ROWS UNBOUNDED PRECEDING) as SumByRows,
SUM(Salary) OVER (ORDER BY Salary RANGE UNBOUNDED PRECEDING) as SumByRange
FROM Sample.Logical
ORDER BY RowID;

RowID,FName,Salary,SumByRows,SumByRange
1,George,800,800,800
2,Sam,950,1750,1750
3,Diane,1100,2850,2850
4,Nicholas,1250,4100,5350
5,Samuel,1250,5350,5350
6,Patricia,1300,6650,6650
7,Brian,1500,8150,8150
8,Thomas,1600,9750,9750
9,Fran,2450,12200,12200
10,Debbie,2850,15050,15050


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

window_rows = Window.orderBy("Salary").rowsBetween(Window.unboundedPreceding, Window.currentRow)
window_range = Window.orderBy("Salary").rangeBetween(Window.unboundedPreceding, Window.currentRow)

res = _sqldf.withColumn("SumByRows", F.sum("Salary").over(window_rows)) \
           .withColumn("SumByRange", F.sum("Salary").over(window_range)) \
           .orderBy("RowID")

res.show()

+-----+-----------+------+---------+----------+
|RowID|      FName|Salary|SumByRows|SumByRange|
+-----+-----------+------+---------+----------+
|    1|     George|   800|      800|       800|
|    2|        Sam|   950|     1750|      1750|
|    3|      Diane|  1100|     2850|      2850|
|    4|   Nicholas|  1250|     4100|      5350|
|    5|     Samuel|  1250|     5350|      5350|
|    6|   Patricia|  1300|     6650|      6650|
|    7|      Brian|  1500|     8150|      8150|
|    8|     Thomas|  1600|     9750|      9750|
|    9|       Fran|  2450|    12200|     12200|
|   10|     Debbie|  2850|    15050|     15050|
|   11|       Mark|  2975|    18025|     18025|
|   12|      James|  3000|    21025|     24025|
|   13|    Cynthia|  3000|    24025|     24025|
|   14|Christopher|  5000|    29025|     29025|
+-----+-----------+------+---------+----------+



In [0]:
#Do tego notatnika dopisz użycie funkcji okienkowych LEAD, LAG, FIRST_VALUE, LAST_VALUE, ROW_NUMBER  

In [0]:
df = spark.sql("""
    SELECT t.AccountId, t.TranDate, l.FName, l.Salary
    FROM Sample.Transactions t
    JOIN Sample.Logical l ON t.AccountId = l.RowID
""")

df.show()

+---------+----------+------+------+
|AccountId|  TranDate| FName|Salary|
+---------+----------+------+------+
|        1|2011-01-28|George|   800|
|        1|2011-01-26|George|   800|
|        1|2011-01-24|George|   800|
|        1|2011-01-22|George|   800|
|        1|2011-01-15|George|   800|
|        1|2011-01-01|George|   800|
|        2|2011-01-29|   Sam|   950|
|        2|2011-01-26|   Sam|   950|
|        2|2011-01-23|   Sam|   950|
|        2|2011-01-22|   Sam|   950|
|        2|2011-01-15|   Sam|   950|
|        2|2011-01-01|   Sam|   950|
|        3|2011-01-30| Diane|  1100|
|        3|2011-01-27| Diane|  1100|
|        3|2011-01-25| Diane|  1100|
|        3|2011-01-22| Diane|  1100|
|        3|2011-01-15| Diane|  1100|
|        3|2011-01-01| Diane|  1100|
+---------+----------+------+------+



In [0]:
windowSpec = Window.partitionBy("AccountId").orderBy("TranDate")

df = df.withColumn("Lead", F.lead("Salary").over(windowSpec)) \
       .withColumn("Lag", F.lag("Salary").over(windowSpec)) \
       .withColumn("FirstValue", F.first("Salary").over(windowSpec)) \
       .withColumn("LastValue", F.last("Salary").over(windowSpec)) \
       .withColumn("RowNum", F.row_number().over(windowSpec))

df.show()

+---------+----------+------+------+----+----+----------+---------+------+
|AccountId|  TranDate| FName|Salary|Lead| Lag|FirstValue|LastValue|RowNum|
+---------+----------+------+------+----+----+----------+---------+------+
|        1|2011-01-01|George|   800| 800|null|       800|      800|     1|
|        1|2011-01-15|George|   800| 800| 800|       800|      800|     2|
|        1|2011-01-22|George|   800| 800| 800|       800|      800|     3|
|        1|2011-01-24|George|   800| 800| 800|       800|      800|     4|
|        1|2011-01-26|George|   800| 800| 800|       800|      800|     5|
|        1|2011-01-28|George|   800|null| 800|       800|      800|     6|
|        2|2011-01-01|   Sam|   950| 950|null|       950|      950|     1|
|        2|2011-01-15|   Sam|   950| 950| 950|       950|      950|     2|
|        2|2011-01-22|   Sam|   950| 950| 950|       950|      950|     3|
|        2|2011-01-23|   Sam|   950| 950| 950|       950|      950|     4|
|        2|2011-01-26|   